### . Accessing ADLS NYPD Dataset using Account Key

In [0]:
#Set up storage account and access key
storage_account_name = "nypdstorageaccount"
storage_account_key = "hBRymEJTK9H//AIZBqbRyWSGZ5H9vWcPnt5ZuXBNYIAt62KEaCStb9erAm2nxyN7hGMDrXF5N/dk+AStOxKWKw=="

#Configure Spark to access ADLS Gen2
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

#Define File Path from the storage account container
adls_path = f"abfss://nypdcontainer@{storage_account_name}.dfs.core.windows.net/"
file_path = adls_path + "nypd.csv"

print("Path:", file_path)

#Reading the csv file
dataframe = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

dataframe.show(5)



Path: abfss://nypdcontainer@nypdstorageaccount.dfs.core.windows.net/nypd.csv
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+-----------------+------------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|           OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|PERP_RACE|X_COORD_CD|Y_COORD_CD|         Latitude|        Longitude|New Georeferenced Column|
+----------+-----------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+-----------------+-----------------+------------------------+
| 298874520| 2025-01-04|  439|LARCENY,GRAND FRO...|  109|       GRAND LARCENY|PL 1553004|         F|          M|              7|                0|

In [0]:
%sql

SHOW CATALOGS


catalog
spark_catalog


In [0]:
dataframe.write.saveAsTable("default.nypd")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5406624853851483>, line 1
----> 1 dataframe.write.saveAsTable("default.nypd")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1857, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1855 if format is not None:
   1856     self.format(format)
-> 1857 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py:1362, in JavaMember.__call__(self, *args)
   1356 command =

In [0]:
from pyspark.sql.functions import col

#Rename columns for Delta compatibility
for old_name in dataframe.columns:
    new_name = old_name.replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "")
    dataframe = dataframe.withColumnRenamed(old_name, new_name)

print(dataframe.columns)


['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude', 'New_Georeferenced_Column']


In [0]:
#Database
spark.sql("CREATE DATABASE IF NOT EXISTS nypd_db")

#Writing delta table format into Hive Metastore
dataframe.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("nypd_db.nypd")

print("The table has been written to the Hive Metastore")


The table has been written to the Hive Metastore


In [0]:
%sql

SELECT * FROM nypd_db.nypd
LIMIT 10;


ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New_Georeferenced_Column
301897930,2025-03-01,268,CRIMINAL MIS 2 & 3,121,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,K,71,0,25-44,M,BLACK,1000539,180250,40.661416,-73.941288,POINT (-73.941288 40.661416)
303761369,2025-03-27,705,"FORGERY,ETC.-MISD.",358,OFFENSES INVOLVING FRAUD,PL 1702000,M,M,25,0,45-64,M,BLACK,1000613,231653,40.80249472,-73.94089675,POINT (-73.94089675 40.80249472)
303552012,2025-03-24,109,"ASSAULT 2,1,UNCLASSIFIED",106,FELONY ASSAULT,PL 1200502,F,B,52,0,45-64,M,BLACK,1012777,255259,40.867265,-73.89686,POINT (-73.89686 40.867265)
302403507,2025-03-11,203,"TRESPASS 3, CRIMINAL",352,CRIMINAL TRESPASS,PL 140100A,M,M,14,3,45-64,M,WHITE,986881,214802,40.756267,-73.990503,POINT (-73.990503 40.756267)
301897944,2025-03-01,759,"PUBLIC ADMINISTATION,UNCLASS M",359,OFFENSES AGAINST PUBLIC ADMINI,PL 1950501,M,Q,115,3,25-44,F,BLACK,1018470,220785,40.7726138877055,-73.8764526037881,POINT (-73.8764526037881 40.7726138877055)
302805110,2025-03-13,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,43,2,25-44,M,BLACK HISPANIC,1022100,238957,40.822487,-73.863245,POINT (-73.863245 40.822487)
302827831,2025-03-15,779,"PUBLIC ADMINISTRATION,UNCLASSI",126,MISCELLANEOUS PENAL LAW,PL 215510B,F,Q,114,0,25-44,M,WHITE,1008041,224166,40.781939,-73.914093,POINT (-73.914093 40.781939)
302162688,2025-03-06,493,"STOLEN PROPERTY-MOTOR VEH 2ND,",111,POSSESSION OF STOLEN PROPERTY,PL 1654505,F,K,67,0,25-44,M,BLACK,996838,177968,40.655157,-73.954631,POINT (-73.954631 40.655157)
303653349,2025-03-26,49,U.S. CODE UNCLASSIFIED,995,FOR OTHER AUTHORITIES,FOA9000049,null,Q,116,0,25-44,M,BLACK,1058380,182237,40.666574,-73.732786,POINT (-73.732786 40.666574)
303384387,2025-03-21,511,"CONTROLLED SUBSTANCE, POSSESSI",235,DANGEROUS DRUGS,PL 2200300,M,K,68,1,25-44,M,WHITE,975608,163701,40.615993603832834,-74.03112773583993,POINT (-74.03112773583993 40.615993603832834)
